# Grading curve

This course has a curve due to [SIPA policy](https://www.sipa.columbia.edu/sipa-education/bulletin/academic-policies/grading):

> SIPA expects an average class GPA of 3.33 (B+) for core courses and those with enrollments exceeding 35 students. The acceptable range for such courses is 3.2 to 3.4.

---

## Scores last updated


In [1]:
from datetime import date

date.today()

datetime.date(2025, 12, 27)

## How course grades work

1. The grades are computed using [the weights listed in the syllabus](https://computing-in-context.afeld.me/#grading).
1. The percentage required for all the letter grades is adjusted up or down as necessary to hit the target GPA range across both sections.
1. The letter grades are [uploaded to SSOL](https://registrar.columbia.edu/content/managing-grades).

The `min_score` column of the [new cutoffs](#new-cutoffs) shows the current minimum `Total` percentage required for each letter grade.

## Methodology

The rest of this notebook shows how the grade cutoffs are computed. <!--**This methodology, the distribution of student percentages, and thus your estimated course grade are subject to change** up until final grades are submitted.-->


In [2]:
MIN_AVG_GPA = 3.2
MAX_AVG_GPA = 3.4

### Load current scores


In [3]:
import pandas as pd

grades = pd.read_csv("/Users/afeld/Downloads/computing-in-context-fall-2025.csv")
grades = grades.sort_values("Final Grade")

<frozen importlib._bootstrap>:491: RuntimeWarning: The global interpreter lock (GIL) has been enabled to load module 'pandas._libs.pandas_parser', which has not declared that it can run safely without the GIL. To override this behavior and keep the GIL disabled (at your own risk), run with PYTHON_GIL=0 or -Xgil=0.


#### Distribution


[Fix rendering:](https://computing-in-context.afeld.me/notebooks.html#jupyter-book)


In [4]:
import plotly.io as pio

pio.renderers.default = "notebook_connected+plotly_mimetype"

In [5]:
import plotly.express as px

fig = px.histogram(
    grades,
    x="Final Grade",
    title="Distribution of the overall grades",
    labels={"Final Grade": "Final Grade (percent)"},
)
fig.update_layout(yaxis_title_text="Number of students")
fig.show()

### Match to letter grades / GPAs

Creating the [Grading Scale](https://www.sipa.columbia.edu/sipa-education/bulletin/academic-policies/grading) in Pandas:


In [6]:
letter_grade_equivalents = pd.DataFrame(
    index=["A", "A-", "B+", "B", "B-", "C+", "C", "C-", "D", "F"],
    data={"gpa": [4.00, 3.67, 3.33, 3.00, 2.67, 2.33, 2.00, 1.67, 1.00, 0.00]},
)

Assign starting minimum scores, roughly based on the Default Canvas Grading Scheme:


In [7]:
letter_grade_equivalents["min_score"] = [94.0, 90.0, 87.0, 84.0, 80.0, 77.0, 74.0, 70.0, 60.0, 0.0]
letter_grade_equivalents

,gpa,min_score
A,4.00,94.0
A-,3.67,90.0
B+,3.33,87.0
B,3.00,84.0
B-,2.67,80.0
C+,2.33,77.0
C,2.00,74.0
C-,1.67,70.0
D,1.00,60.0
F,0.00,0.0


### Adjust cutoffs

Raise or lower the minimum scores for each grade (not including F) until the average GPA is in the acceptable range.


In [8]:
# merge_asof() needs columns sorted ascending
orig_grade_cutoffs = letter_grade_equivalents.sort_values(by="min_score")
grade_cutoffs = orig_grade_cutoffs.copy()

grades_to_adjust = grade_cutoffs.index != "F"

adjustment = 0
STEP_SIZE = 0.1

while True:
    grade_cutoffs.loc[grades_to_adjust, "min_score"] = orig_grade_cutoffs[grades_to_adjust]["min_score"] + adjustment

    # make the letter grades a column so they show up in the merged DataFrame
    grade_cutoffs_with_letters = grade_cutoffs.reset_index().rename(columns={"index": "letter_grade"})

    # find the letter grade / GPA for each student
    adjusted_grades = pd.merge_asof(
        grades,
        grade_cutoffs_with_letters,
        left_on="Final Grade",
        right_on="min_score",
        direction="backward",
    )

    new_mean = adjusted_grades["gpa"].mean()
    print(f"Adjustment: {adjustment:+.1f}, Average: {new_mean:.3f}")

    # check if we've hit the target range
    if MIN_AVG_GPA <= new_mean < MAX_AVG_GPA:
        # success
        break
    elif new_mean >= MAX_AVG_GPA:
        # raise
        adjustment += STEP_SIZE
    else:  # new_mean < MIN_AVG_GPA:
        # lower
        adjustment -= STEP_SIZE

Adjustment: +0.0, Average: 3.519
Adjustment: +0.1, Average: 3.511
Adjustment: +0.2, Average: 3.511
Adjustment: +0.3, Average: 3.503
Adjustment: +0.4, Average: 3.499
Adjustment: +0.5, Average: 3.499
Adjustment: +0.6, Average: 3.499
Adjustment: +0.7, Average: 3.487
Adjustment: +0.8, Average: 3.480
Adjustment: +0.9, Average: 3.480
Adjustment: +1.0, Average: 3.480
Adjustment: +1.1, Average: 3.472
Adjustment: +1.2, Average: 3.460
Adjustment: +1.3, Average: 3.452
Adjustment: +1.4, Average: 3.444
Adjustment: +1.5, Average: 3.432
Adjustment: +1.6, Average: 3.424
Adjustment: +1.7, Average: 3.409
Adjustment: +1.8, Average: 3.405
Adjustment: +1.9, Average: 3.397


Confirm the A cutoff is still achievable:


In [9]:
assert grade_cutoffs.at["A", "min_score"] <= 100  # type: ignore

#### New cutoffs


In [10]:
grade_cutoffs.sort_values("min_score", ascending=False)

,gpa,min_score
A,4.00,95.9
A-,3.67,91.9
B+,3.33,88.9
B,3.00,85.9
B-,2.67,81.9
C+,2.33,78.9
C,2.00,75.9
C-,1.67,71.9
D,1.00,61.9
F,0.00,0.0


### Check results


Double-check the new average is in line with policy:


In [11]:
assert MIN_AVG_GPA <= new_mean < MAX_AVG_GPA, f"{new_mean} not in acceptable range"

new_mean

np.float64(3.3969411764705884)

In [12]:
fig = px.histogram(adjusted_grades, x="letter_grade", title="Distribution of letter grades")
fig.update_layout(yaxis_title_text="Number of students")
fig.show()

## Export

Format needed by SSOL.


In [13]:
ssol = adjusted_grades.rename(columns={"SID": "UNI", "letter_grade": "grade"})
ssol = ssol[["UNI", "grade"]].sort_values("UNI")

ssol.to_csv("tmp/grades.csv", index=False)
print("Done")

Done
